### Imports


In [2]:
import gradio as gr
import os
import time
import google.generativeai as genai

### Initial configuration


In [13]:
genai.configure(api_key=os.environ["API_KEY"])
if not os.environ["API_KEY"]:
    print("API key not found. Please set the API_KEY environment variable.")
    exit()
else:
    print("API key found.")

model = genai.GenerativeModel(
    "gemini-1.5-flash-latest",
    system_instruction=f"""
    Respond in style of a mushroom expert chatbot.
    The messages should be only about mushrooms. 
    if the user asks about something else,you can ask that they ask about mushrooms instead.
    keep the chat focused and compact and avoid chit-chat.
    when asked to identify a mushroom, only provide the answer if you are confident, otherwise mention what you think and that you are not sure.
    """,
)

API key found.


### Gradio Implementation


In [29]:
# Check if 'demo' is running and close it
if "demo" in locals() and demo.is_running:
    demo.close()

# Start a new chat session with the model
chat = model.start_chat(history=[])

# Define the JSON request message
json_request_message = """
Produce a JSON response with the following fields:
- common_name
- genus
- confidence (of the prediction)
- visible (what parts of the mushrooms are visible in the image, from the set {cap, hymenium, stipe})
- color (of the mushroom in the picture)
- edible (boolean indicating the edibility of the mushroom)

Example JSON response:
{
  "common_name": "Inkcap",
  "genus": "Coprinus",
  "confidence": 0.5,
  "visible": ["cap", "hymenium", "stipe"],
  "color": "orange",
  "edible": true
}
"""

# Define the summary prompt
summary_prompt = """
Provide a detailed summary of the image based on the previously provided JSON data.
Mention all of the JSON properties except confidence.
Do not mention the JSON file itself in the response.
"""


def response(input_data, history):
    global chat
    text_input = input_data.get("text", "")
    files = input_data.get("files", [])

    # Check if a file is provided
    if files and isinstance(files, list) and len(files) > 0:
        file_path = files[0]["path"]
        text_input = text_input or summary_prompt

        try:
            uploaded_file = genai.upload_file(path=file_path)

            if text_input == summary_prompt:
                # Generate JSON response based on the image
                json_response = model.generate_content(
                    [uploaded_file, json_request_message]
                )
                print("\n-- JSON BASED ON IMAGE --")
                print(json_response.text)
                print("-------------------------")

                # Append the JSON response to the chat history
                chat.history.append({"parts": [json_response.text], "role": "model"})

            # Send a message to the chat with the uploaded file and text input
            gemini_response = chat.send_message(
                {"parts": [uploaded_file, text_input], "role": "user"}, stream=True
            )

        except Exception as e:
            print(f"Error uploading file: {e}")
            print(e)
            error_message = (
                "An error occurred while uploading the file. "
                "Please try again with a different input."
            )
            for i in range(len(error_message)):
                time.sleep(0.01)
                yield error_message[: i + 1]
            return
    else:
        # If no file is provided, process only the text input
        gemini_response = chat.send_message(
            {"parts": [text_input], "role": "user"}, stream=True
        )

    # Resolve the response and print the chat history
    gemini_response.resolve()

    try:
        # Check for safety ratings in the response
        dangerous = any(
            rating.probability != 1
            for rating in gemini_response.candidates[0].safety_ratings
        )

        if dangerous:
            # Rewind the chat history if the content is dangerous
            chat.rewind()
            warning_message = (
                "The model has detected that the content of the response may be harmful or inappropriate. "
                "Please try again with a different input."
            )
            for i in range(len(warning_message)):
                time.sleep(0.007)
                yield warning_message[: i + 1]
        else:
            # Stream the response text to the user
            for i in range(len(gemini_response.text)):
                time.sleep(0.01)
                yield gemini_response.text[: i + 1]
    except Exception as e:
        print(f"Error processing response: {e}")
        error_message = (
            "An error has occurred. Please try again with a different input."
        )
        for i in range(len(error_message)):
            time.sleep(0.007)
            yield error_message[: i + 1]
        chat.rewind()
        return


# Create and launch the Gradio interface
demo = gr.ChatInterface(fn=response, multimodal=True, theme="Default")

demo.launch(share=True)

Closing server running on port: 7870
Running on local URL:  http://0.0.0.0:7870
Running on public URL: https://e4882dff50b063585b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)



-- JSON BASED ON IMAGE --
{
  "common_name": "Chanterelle",
  "genus": "Cantharellus",
  "confidence": 0.9,
  "visible": ["cap", "hymenium", "stipe"],
  "color": "orange",
  "edible": true
} 

-------------------------
